In [8]:
import inspect
from functools import wraps
import time
import datetime

In [24]:
def logger(fn):
    @wraps(fn)
    def wrapper(*args,**kwargs):
        start = datetime.datetime.now()
        ret = fn(*args,**kwargs)
        delta = (datetime.datetime.now()-start).total_seconds()
        
        print(fn.__name__,delta)
        return ret
    return wrapper

In [33]:
def mag_cache(duration = 5):   
    def _mag_cache(fn):
        local_cache = {}
        @wraps(fn)
        def wrapper(*args,**kwargs):

            def _clear_expire():
                expire_keys = []
                for k ,(_,stamp) in local_cache.items():
                    if datetime.datetime.now().timestamp() - stamp > duration: #handling time: delete element exceeding duration seconds
                        expire_keys.append(k)
                for k in expire_keys:
                    local_cache.pop(k)

            def _make_key(args,kwargs):  
                sig = inspect.signature(fn)
                params = sig.parameters ## ordered dict
                param_dict = {}

                param_dict.update(zip(params.keys(),args)) #positional parameters
                param_dict.update(kwargs)          #key-word parameters

                for k,v in params.items():   ##default parameters
                    if k not in param_dict:
                        param_dict[k] = v.default

                key = tuple(sorted(param_dict.items()))
                return key

            _clear_expire()
            key = _make_key(args,kwargs)

            if key not in local_cache:
                local_cache[key] = fn(*args,**kwargs),datetime.datetime.now().timestamp()

            return local_cache[key]
        return wrapper
    return _mag_cache

@logger 
@mag_cache()
def add(x=4,y=5):
    time.sleep(3)
    return x+y

In [34]:
add()

add 3.003219


(9, 1567525285.491417)

In [35]:
add(4,5)

add 5.3e-05


(9, 1567525285.491417)

In [36]:
add(x=4,y=5)

add 5.7e-05


(9, 1567525285.491417)

In [31]:
add(y=5,x=4)

add 7e-05


(9, 1567524580.649705)